# Minimization of metabolic adjustment (MOMA)

MOMA allows you to find a feasible solution of the model that is closest (in
an Euclidean metric) to a reference solution. Often this gives a realistic
estimate of the organism behavior that has undergone a radical change (such
as a gene knockout) that prevents it from metabolizing optimally, but the
rest of the metabolism has not yet adjusted to compensate for the change.

The original description of MOMA is by [Segre, Vitkup, and Church, "Analysis
of optimality in natural and perturbed metabolic networks", Proceedings of the
National Academy of Sciences, 2002](https://doi.org/10.1073/pnas.232349399).

As always, let's start with downloading a model.

In [1]:
!isfile("e_coli_core.xml") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA

model = load_model(StandardModel, "e_coli_core.xml")

Metabolic model of type StandardModel
sparse([41, 23, 51, 67, 61, 65, 1, 7, 19, 28  …  72, 3, 8, 33, 57, 66, 31, 45, 46, 57], [1, 2, 2, 2, 3, 3, 4, 4, 4, 4  …  93, 94, 94, 94, 94, 94, 95, 95, 95, 95], [-1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0  …  1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


MOMA analysis requires solution of a quadratic model, we will thus use OSQP as the main optimizer.

In [2]:
using OSQP

We will need a reference solution, which represents the original state of the
organism before the change.

In [3]:
reference_flux = flux_balance_analysis_dict(
    model,
    OSQP.Optimizer;
    modifications = [silence, change_optimizer_attribute("polish", true)],
)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.00669695
  "R_ACONTb"      => 6.09079
  "R_TPI"         => 7.50412
  "R_SUCOAS"      => -5.19078
  "R_GLNS"        => 0.223103
  "R_EX_pi_e"     => -3.26725
  "R_PPC"         => 2.45788
  "R_O2t"         => 22.1552
  "R_G6PDH2r"     => 5.03314
  "R_TALA"        => 1.51888
  "R_PPCK"        => 0.0032309
  "R_EX_lac__D_e" => -0.019119
  "R_PGL"         => 5.03317
  "R_H2Ot"        => -29.5302
  "R_GLNabc"      => 0.014039
  "R_EX_co2_e"    => 23.2225
  "R_EX_gln__L_e" => -0.0140251
  "R_EX_nh4_e"    => -4.78831
  "R_MALt2_2"     => 0.00736307
  ⋮               => ⋮

As the change here, we manually knock out CYTBD reaction:

In [4]:
changed_model = change_bound(model, "R_CYTBD", lower = 0.0, upper = 0.0);

Now, let's find a flux that minimizes the organism's metabolic adjustment for
this model:

In [5]:
flux_summary(
    minimize_metabolic_adjustment_analysis_dict(
        changed_model,
        reference_flux,
        OSQP.Optimizer;
        modifications = [silence, change_optimizer_attribute("polish", true)],
    ),
)

Biomass
  R_BIOMASS_Ecoli_core_w_GAM: 0.062
Import
  R_EX_glc__D_e:              -9.2156
  R_EX_nh4_e:                 -1.3021
  R_EX_pi_e:                  -0.2281
  R_EX_fum_e:                 -0.0001
  R_EX_mal__L_e:              -0.0001
  R_EX_gln__L_e:              -0.0
Export
  R_EX_fru_e:                 0.0
  R_EX_for_e:                 0.0
  R_EX_o2_e:                  0.0001
  R_EX_pyr_e:                 0.0002
  R_EX_lac__D_e:              0.0002
  R_EX_akg_e:                 0.0002
  R_EX_ac_e:                  0.0004
  R_EX_glu__L_e:              0.9641
  R_EX_acald_e:               1.728
  R_EX_co2_e:                 5.2146
  R_EX_h2o_e:                 5.7974
  R_EX_succ_e:                5.9278
  R_EX_etoh_e:                7.7251
  R_EX_h_e:                   15.0284


For illustration, you can compare the result to the flux that is found by
simple optimization:

In [6]:
flux_summary(
    flux_balance_analysis_dict(
        changed_model,
        OSQP.Optimizer;
        modifications = [silence, change_optimizer_attribute("polish", true)],
    ),
)

Biomass
  R_BIOMASS_Ecoli_core_w_GAM: 0.2057
Import
  R_EX_glc__D_e:              -10.0152
  R_EX_h2o_e:                 -4.5202
  R_EX_nh4_e:                 -1.0837
  R_EX_pi_e:                  -0.7565
  R_EX_glu__L_e:              -0.0175
  R_EX_akg_e:                 -0.0159
  R_EX_gln__L_e:              -0.0102
  R_EX_pyr_e:                 -0.0079
  R_EX_fru_e:                 -0.0076
  R_EX_lac__D_e:              -0.0069
  R_EX_fum_e:                 -0.0055
  R_EX_succ_e:                -0.0054
  R_EX_mal__L_e:              -0.0043
  R_EX_acald_e:               -0.0035
  R_EX_o2_e:                  -0.0001
Export
  R_EX_co2_e:                 5.5013
  R_EX_ac_e:                  5.5395
  R_EX_etoh_e:                11.3931
  R_EX_for_e:                 12.3487
  R_EX_h_e:                   21.8819


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*